# 런타임에 체인 내부 구성하기

개발자는 종종 여러 가지 다양한 방식으로 작업을 수행하는 과정이나 실험내용을 최종 사용자(End User)에게 노출하고 싶어 할 때가 있습니다.

이러한 경험을 최대한 쉽게 만들기 위하여 다음의 2가지 메서를 활용할 수 있습니다.

- 첫째, `configurable_fields` 메서드입니다. 이 메서드를 통해 실행 가능한 객체의 특정 필드를 구성할 수 있습니다.

- 둘째, `configurable_alternatives` 메서드입니다. 이 메서드를 사용하면 런타임 중에 설정할 수 있는 특정 실행 가능한 객체에 대한 대안을 나열할 수 있습니다.


## configurable_fields

`configurable_fields` 는 시스템의 설정 값을 정의하는 필드를 의미합니다. 이러한 설정 필드는 시스템의 동작을 제어하고 사용자의 요구사항에 맞게 시스템을 구성하는 데 사용됩니다. `configurable_fields` 를 통해 사용자는 시스템의 다양한 측면을 조정할 수 있으며, 이를 통해 시스템의 성능과 기능을 최적화할 수 있습니다.

일반적으로 `configurable_fields` 는 설정 파일, 환경 변수, 또는 사용자 인터페이스를 통해 제공되며, 시스템 관리자나 개발자가 이를 관리합니다.

`configurable_fields`의 설정할 수 있는 파라미터의 예시로는 데이터베이스 연결 정보, 로깅 설정, 보안 옵션, 성능 튜닝 매개변수 등이 있습니다. 이러한 필드를 적절히 설정함으로써 시스템을 효과적으로 구성하고 관리할 수 있습니다.


## 동적 속성 지정

`ChatOpenAI` 을 사용할 때, 우리는 `model_name` 와 같은 설정을 조정할 수 있습니다.

`model_name` 은 GPT 의 버전을 명시할 때 사용하는 속성입니다. 예를 들어, `gpt-3.5-turbo`, `gpt-4` 등을 설정하여 모델을 선택할 수 있습니다.

만약, 고정된 `model_name` 이 아닌 동적으로 모델을 지정하고 싶을 때는 다음과 같이 `ConfigurableField` 를 활용하여 동적으로 설정할 수 있는 속성 값으로 변환할 수 있습니다.


In [ ]:
%pip install -qU langchain langchain-openai

`configurable_fields` 메서드를 사용하여 `model_name` 속성을 동적 구성 가능한 필드로 지정합니다.


In [5]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    model_name=ConfigurableField(  # model_name 은 원래 ChatOpenAI 의 필드입니다.
        id="gpt_version",  # model_name 의 id 를 설정합니다.
        name="Version of GPT",  # model_name 의 이름을 설정합니다.
        # model_name 의 설명을 설정합니다.
        description="Official model name of GPTs. ex) gpt-3.5-turbo, gpt-4",
    )
)

`model.invoke()` 호출시 `config={"configurable": {"키": "값"}}` 형식으로 동적 지정할 수 있습니다.


In [11]:
model.invoke(
    "대한민국의 수도는 어디야?",
    # gpt_version 을 gpt-3.5-turbo 로 설정합니다.
    config={"configurable": {"gpt_version": "gpt-3.5-turbo"}},
)

AIMessage(content='대한민국의 수도는 서울이다.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 22, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

이번에는 `gpt-4` 모델을 사용해보겠습니다. 출력에 바뀐 모델을 확인하세요.


In [12]:
model.invoke(
    # gpt_version 을 gpt-4 로 설정합니다.
    "대한민국의 수도는 어디야?",
    config={"configurable": {"gpt_version": "gpt-4"}},
)

AIMessage(content='대한민국의 수도는 서울입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 22, 'total_tokens': 37}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})

`model` 객체의 `with_config()` 메서드를 사용하여 `configurable` 매개변수를 설정할 수도 있습니다. 이전과 동작하는 방식은 동일합니다.


In [14]:
model.with_config(configurable={"gpt_version": "gpt-4-turbo-preview"}).invoke(
    "대한민국의 수도는 어디야?"
)

AIMessage(content='대한민국의 수도는 서울입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 22, 'total_tokens': 37}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': 'fp_a7daf7c51e', 'finish_reason': 'stop', 'logprobs': None})

또한 이 함수를 체인의 일부로 사용할 때에도 동일한 방식으로 활용할 수 있습니다.


In [15]:
# 템플릿에서 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{x} 보다 큰 위의 난수를 선택합니다.")
chain = (
    prompt | model
)  # 프롬프트와 모델을 연결하여 체인을 생성합니다. 프롬프트의 출력이 모델의 입력으로 전달됩니다.

In [16]:
chain.invoke({"x": 0})  # 체인을 호출하고 입력 변수 "x"에 0을 전달합니다.

AIMessage(content='5', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 21, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [18]:
# 체인을 호출할 때 설정을 지정하여 체인을 호출할 수 있습니다.
chain.with_config(configurable={"gpt_version": "gpt-4"}).invoke({"x": 0})

AIMessage(content='죄송합니다만, 제가 난수를 생성하거나 선택하는 기능은 없습니다. 이 기능은 프로그래밍 언어나 소프트웨어를 사용해야 합니다. 예를 들어, 파이썬에서는 random 모듈을 사용하여 난수를 생성할 수 있습니다.', response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 21, 'total_tokens': 105}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})

## HubRunnable: LangChain Hub의 설정 변경

`HubRunnable` 을 사용하면 Hub 에 등록된 프롬프트의 전환을 용이하게 합니다.


In [35]:
from langchain.runnables.hub import HubRunnable

prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    # 소유자 저장소 커밋을 설정하는 ConfigurableField
    owner_repo_commit=ConfigurableField(
        # 필드의 ID
        id="hub_commit",
        # 필드의 이름
        name="Hub Commit",
        # 필드에 대한 설명
        description="The Hub commit to pull from",
    )
)

별도의 `with_config` 지정 없이 `prompt.invoke()` 메서드를 호출하면 처음 설정한 `"rlm/rag-prompt"` hub 에 등록된 프롬프트를 pull 하여 가져옵니다.


In [37]:
# prompt 객체의 invoke 메서드를 호출하여 "question"과 "context" 매개변수를 전달합니다.
prompt.invoke({"question": "Hello", "context": "World"})

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: Hello \nContext: World \nAnswer:")])

In [43]:
prompt.with_config(
    # hub_commit 을 teddynote/simple-summary-korean 으로 설정합니다.
    configurable={"hub_commit": "teddynote/simple-summary-korean"}
).invoke({"context": "Hello"})

ChatPromptValue(messages=[HumanMessage(content='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: Hello\n\nSUMMARY:')])

## Configurable Alternatives: Runnable 객체 자체의 대안 설정

런타임에 설정할 수 있는 Runnable 에 대한 대안을 구성합니다.

**구성 가능한 대안들**

`ChatAnthropic` 의 구성 가능한 언어 모델은 다양한 작업과 컨텍스트에 적용할 수 있는 유연성을 제공합니다.

모델의 동작을 제어하기 위해 다음과 같은 구성 옵션을 사용할 수 있습니다.

- `model`: 사용할 기본 언어 모델을 지정합니다. 예를 들어, "claude-v1", "claude-instant-v1" 등이 있습니다.

- `temperature`: 0에서 1 사이의 값으로, 샘플링의 무작위성을 제어합니다. 값이 낮을수록 더 결정적이고 반복적인 출력이 생성되며, 값이 높을수록 더 다양하고 창의적인 출력이 생성됩니다.

- `max_tokens_to_sample`: 생성할 최대 토큰 수를 지정합니다.

- `stop_sequences`: 지정된 문자열 시퀀스 중 하나가 생성되면 생성을 중지합니다.

- `top_k`: 샘플링에 고려할 최상위 확률의 토큰 수를 지정합니다.

- `top_p`: 누적 확률 질량의 상위 부분에서 토큰을 샘플링합니다.

이러한 구성 옵션을 조정하여 모델의 동작을 작업에 맞게 최적화할 수 있습니다.


### LLM 객체의 대안(alternatives) 설정 방법

LLM(Large Language Model)을 활용하여 이를 수행하는 방법을 살펴보겠습니다.

[참고]

- `ChatAnthropic` 을 사용하기 위하여 API KEY를 발급받아 설정해야합니다.
- 아래 주석을 해제하고 API KEY를 설정하거나, `.env` 파일에 설정합니다.


In [9]:
# import os

# os.environ["ANTHROPIC_API_KEY"] = "ANTHROPIC API KEY를 입력합니다."

In [10]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatAnthropic
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatAnthropic(temperature=0).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    # 이 키를 지정하면 위에서 초기화된 기본 LLM(ChatAnthropic)이 사용됩니다.
    default_key="anthropic",
    # 'openai'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI()`와 동일합니다.
    openai=ChatOpenAI(),
    # 'gpt4'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4")`와 동일합니다.
    gpt4=ChatOpenAI(model="gpt-4"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)
prompt = PromptTemplate.from_template("{topic} 에 대해 간단히 설명해주세요.")
chain = prompt | llm

`chain.invoke()` 메서드를 기본 LLM 인 `ChatAnthropic` 을 활용한 체인을 호출합니다.


In [12]:
# Anthropic을 기본으로 호출합니다.
chain.invoke({"topic": "뉴진스"})

AIMessage(content=" 뉴진스(New Jeans)는 2022년 데뷔한 걸그룹입니다.\n\n- 소속사는 하이브엔터테인먼트\n- 멤버는 민지, 해린, 다니엘, 해원, 혜인 총 5명\n- 2022년 8월 1일 데뷔 싱글 앨범 'New Jeans'로 데뷔\n- 타이틀 곡 'Attention'이 큰 인기를 끌며 차트 1위를 기록\n- 10대 후반~20대 초반 멤버로 구성되어 있어 MZ 세대의 지지를 받고 있음\n- 트렌디하고 개성 있는 패션과 음악 스타일이 특징\n\n뉴진스는 청량하고 ")

`chain.with_config(configurable={"llm": "모델"})`를 사용하여 사용할 `llm`으로 다른 모델을 지정할 수 있습니다.


In [15]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "뉴진스"})

AIMessage(content="뉴진스(New Jin's)는 한국의 한 혼밥 레스토랑 체인으로, 한 그룹이 함께 앉아 식사를 즐길 수 있는 테이블을 제공하는 것이 특징입니다. 메뉴는 주로 한식과 중식을 조합한 요리로 구성되어 있으며, 가격대도 합리적인 편이라 인기가 있는 곳입니다. 식사를 하면서 소셜 미디어에 사진을 찍는 것을 즐기는 사람들에게 인기가 있습니다.", response_metadata={'token_usage': {'completion_tokens': 168, 'prompt_tokens': 27, 'total_tokens': 195}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})

체인의 설정을 변경하여 사용할 언어 모델을 `gpt4` 로 지정합니다.


In [17]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "gpt4"}).invoke({"topic": "뉴진스"})

AIMessage(content='뉴진스는 대한민국의 의료기기 제조 기업입니다. 주로 의료 영상 진단기기를 제조 및 판매하며, 특히 초음파 진단기기에 특화되어 있습니다. 또한, 의료용 소프트웨어 개발과 서비스 제공 등 다양한 분야에서 활동하고 있습니다.', response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 27, 'total_tokens': 148}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})

체인의 설정을 변경하여 사용할 언어 모델을 `anthropic` 로 지정합니다.


In [19]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "anthropic"}).invoke({"topic": "뉴진스"})

AIMessage(content=" 뉴진스(New Jeans)는 2022년 데뷔한 걸그룹입니다.\n\n- 소속사는 하이브엔터테인먼트\n- 멤버는 민지, 해린, 다니엘, 해원, 혜인 총 5명\n- 2022년 8월 1일 데뷔 싱글 앨범 'New Jeans'로 데뷔\n- 타이틀 곡 'Attention'이 큰 인기를 끌며 차트 1위를 기록\n- 10대 후반~20대 초반 멤버로 구성된 틴크러시 콘셉트\n- 트렌디하고 개성 있는 패션과 음악 스타일이 특징\n\n뉴진스는 청량하고 트렌디")

## 프롬프트의 대안 설정 방법

프롬프트도 이전의 LLM 대안 설정 방법과 유사한 작업을 수행할 수 있습니다.


In [29]:
# 언어 모델을 초기화하고 temperature를 0으로 설정합니다.
llm = ChatOpenAI(temperature=0)

prompt = PromptTemplate.from_template(
    "{country} 의 수도는 어디야?"  # 기본 프롬프트 템플릿
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    ConfigurableField(id="prompt"),
    # 기본 키를 설정합니다.
    default_key="capital",
    # 'area'이라는 새로운 옵션을 추가합니다.
    area=PromptTemplate.from_template("{country} 의 면적은 얼마야?"),
    # 'population'이라는 새로운 옵션을 추가합니다.
    population=PromptTemplate.from_template("{country} 의 인구는 얼마야?"),
    # 'eng'이라는 새로운 옵션을 추가합니다.
    eng=PromptTemplate.from_template("{input} 을 영어로 번역해주세요."),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)

# 프롬프트와 언어 모델을 연결하여 체인을 생성합니다.
chain = prompt | llm

아무런 설정 변경이 없다면 기본 프롬프트가 입력됩니다.


In [30]:
# config 변경 없이 체인을 호출합니다.
chain.invoke({"country": "대한민국"})

AIMessage(content='대한민국의 수도는 서울이다.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 23, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})

`with_config` 로 다른 프롬프트를 호출합니다.


In [31]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "area"}).invoke({"country": "대한민국"})

AIMessage(content='대한민국의 총 면적은 약 100,363km² 입니다.', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 25, 'total_tokens': 50}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})

In [32]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "population"}).invoke({"country": "대한민국"})

AIMessage(content='2021년 7월 기준 대한민국의 인구는 약 5천 130만 명입니다.', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 24, 'total_tokens': 58}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})

이번에는 `eng` 프롬프트를 사용하여 번역을 요청합니다. 이때 전달할 입력 변수는 `input`입니다.


In [35]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "eng"}).invoke({"input": "사과는 맛있어!"})

AIMessage(content='Apples are delicious!', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 28, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

## 프롬프트 & LLM 모두 변경

프롬프트와 LLM을 사용하여 여러 가지 사항을 구성할 수 있습니다.

다음은 프롬프트와 LLM을 모두 사용하여 이를 수행하는 예시입니다.


In [47]:
llm = ChatAnthropic(temperature=0).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    # 이 키를 지정하면 위에서 초기화된 기본 LLM(ChatAnthropic)이 사용됩니다.
    default_key="anthropic",
    # 'openai'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI()`와 동일합니다.
    openai=ChatOpenAI(),
    # 'gpt4'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4")`와 동일합니다.
    gpt4=ChatOpenAI(model="gpt-4"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)

prompt = PromptTemplate.from_template(
    "{company} 에 대해서 20자 이내로 설명해 줘."  # 기본 프롬프트 템플릿
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    ConfigurableField(id="prompt"),
    # 기본 키를 설정합니다.
    default_key="description",
    # 'founder'이라는 새로운 옵션을 추가합니다.
    founder=PromptTemplate.from_template("{company} 의 창립자는 누구인가요?"),
    # 'competitor'이라는 새로운 옵션을 추가합니다.
    competitor=PromptTemplate.from_template("{company} 의 경쟁사는 누구인가요?"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)
chain = prompt | llm

In [49]:
# with_config 로 설정 값을 지정하여 구성할 수 있습니다.
chain.with_config(configurable={"prompt": "founder", "llm": "openai"}).invoke(
    # 사용자가 제공한 회사에 대한 처리를 요청합니다.
    {"company": "애플"}
)

AIMessage(content='애플의 창립자는 스티브 잡스(Steve Jobs), 스티브 워즈니악(Steve Wozniak), 론 웨인(Ronald Wayne)입니다. 하지만 론 웨인은 이후 회사에서 물러나게 되었고, 스티브 잡스와 스티브 워즈니악이 애플을 성공적으로 이끌어갔습니다.', response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 27, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [50]:
# 하나만 구성하려는 경우
chain.with_config(configurable={"llm": "anthropic"}).invoke({"company": "애플"})

AIMessage(content=' 애플은 스마트폰과 태블릿, 컴퓨터 등의 전자기기를 제조하는 미국의 회사입니다.')

In [53]:
# 하나만 구성하려는 경우
chain.with_config(configurable={"prompt": "competitor"}).invoke({"company": "애플"})

AIMessage(content=' 애플의 주요 경쟁사에는 다음이 포함됩니다:\n\n- 삼성: 스마트폰과 태블릿에서 애플의 가장 큰 경쟁사. 갤럭시 시리즈가 아이폰과 경쟁합니다.\n\n- 구글: 안드로이드 운영 체제를 통해 스마트폰 시장에서 애플과 경쟁합니다. 또한 구글의 픽셀 스마트폰이 애플 제품과 직접 경쟁합니다.  \n\n- 마이크로소프트: 윈도우 기반 PC와 서피스 제품군을 통해 애플의 맥북과 아이패드 제품군과 경쟁합니다.\n\n- Huawei: 중국의 큰 스마트폰 제')

In [54]:
# 하나만 구성하려는 경우
chain.invoke({"company": "애플"})

AIMessage(content=' 애플은 스마트폰과 태블릿, 컴퓨터 등의 전자기기를 제조하는 미국의 회사입니다.')

## 설정 저장

구성된 체인을 별도의 객체로 쉽게 저장할 수 있습니다. 예를 들어, 특정 작업을 위해 사용자 정의된 체인을 구성한 후, 이를 재사용 가능한 객체로 저장함으로써 향후 유사한 작업에서 손쉽게 활용할 수 있습니다.


In [56]:
# with_config 로 설정을 변경하여 생성한 체인을 별도의 변수에 저장합니다.
gpt4_competitor_chain = chain.with_config(
    configurable={"llm": "gpt4", "prompt": "competitor"}
)

In [57]:
# 체인을 호출합니다.
gpt4_competitor_chain.invoke({"company": "애플"})

AIMessage(content='애플의 주요 경쟁사에는 삼성, 구글, 마이크로소프트, 아마존, 페이스북 등이 있습니다. 이들 회사는 스마트폰, 태블릿, 컴퓨터, 소프트웨어, 클라우드 서비스, 사회 네트워킹 등 다양한 분야에서 애플과 경쟁하고 있습니다.', response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 27, 'total_tokens': 151}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})